## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2
### Дисциплина: Интеллектуальный анализ данных
#### Поиск ассоциативных правил

***


### Студент:  Фаик Карим Яссерович

### Вариант №_17

Вариант 17 

Алгоритм: FPMax 

День недели (поле order_dow таблицы orders): “3” 

Код департамента (поле department_id таблицы products): “6” 

Запрос: определить список товаров, которые не были приобретены ни разу 

Показатель оценки ассоциативных правил: лифт (lift) 

## 1. При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import sqlite3
conn = sqlite3.connect('instacart.db')
cursor = conn.cursor()

## 2. Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме, количество транзакций (покупок) и определите количество товаров (столбец product_id) в транзакциях датафрейма.

In [3]:
departments = pd.read_sql_query("SELECT * FROM departments", conn)
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [4]:
products = pd.read_sql_query("SELECT * FROM products", conn)
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [5]:
order_products__train = pd.read_sql_query(
"""
SELECT order_products__train.*
FROM order_products__train
INNER JOIN orders ON order_products__train.order_id = orders.order_id
INNER JOIN products ON order_products__train.product_id = products.product_id
WHERE orders.order_dow = 3 AND products.department_id = 6
ORDER BY order_id DESC
""", conn)
order_products__train

,order_id,product_id,add_to_cart_order,reordered
0,999926,1540,12,0
1,998420,2658,8,0
2,989699,12748,17,0
3,988381,16876,1,1
4,987722,28568,3,0
...,...,...,...,...
1337,1016198,29919,8,1
1338,1012833,23067,2,1
1339,1010269,15622,3,1
1340,1008624,766,8,1


In [6]:
len(order_products__train)

1342

In [7]:
len(order_products__train['order_id'].unique()) 

1075

In [8]:
order_products__train.order_id.value_counts()

order_id
2931168    9
2053855    4
1107305    4
2880413    4
3384821    4
          ..
3050020    1
304265     1
3040595    1
3039209    1
1002534    1
Name: count, Length: 1075, dtype: int64

## 3. Выполните к датафрейму запрос, указанный в индивидуальном задании.

In [9]:
conn = sqlite3.connect('instacart.db')
query = """
SELECT *
FROM products p
LEFT JOIN order_products__train o on o.product_id = p.product_id
WHERE o.product_id is null
"""
order_products_count = pd.read_sql_query(query, conn)

print("Список товаров, которые не были приобретены ни разу:")
order_products_count

Список товаров, которые не были приобретены ни разу:


,product_id,product_name,aisle_id,department_id,order_id,product_id,add_to_cart_order,reordered
0,6,Dry Nose Oil,11,11,None,None,None,None
1,17,Rendered Duck Fat,35,12,None,None,None,None
2,19,Gluten Free Quinoa Three Cheese & Mushroom Blend,63,9,None,None,None,None
3,20,Pomegranate Cranberry & Aloe Vera Enrich Drink,98,7,None,None,None,None
4,24,Tri-Vi-Sol® Vitamins A-C-and D Supplement Drop...,47,11,None,None,None,None
...,...,...,...,...,...,...,...,...
10560,49666,Ginger Snaps Snacking Cookies,61,19,None,None,None,None
10561,49669,K Cup Dark Blend,100,21,None,None,None,None
10562,49673,Stone Baked Multi Grain Artisan Rolls,129,1,None,None,None,None
10563,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,None,None,None,None


## 4. Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных транзакцию с наибольшим количеством товаров и выведите ее на экран.  

In [10]:
trans_temp = pd.merge(order_products__train, products, on="product_id")[["order_id", "product_name"]]
transactions = trans_temp.groupby('order_id', as_index=True)
transactions_db = []
for order_id in trans_temp["order_id"].unique():
    transactions_db.append(list(transactions["product_name"].get_group(order_id)))
transactions_db

[['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Organic Southwest Taco Seasoning'],
 ['Unsweetened Coconut Milk'],
 ['Unsweetened Coconut Milk',
  'Tofu Shirataki Fettuccine Noodle Substitute',
  'Tofu Shirataki Angel Hair Noodle Substitute'],
 ['Unsweetened Coconut Milk'],
 ['Unsweetened Coconut Milk'],
 ['Mexican Style Hominy'],
 ['Mexican Style Hominy', 'Verde Salsa', 'Mild Enchilada Sauce'],
 ['Mexican Style Hominy'],
 ['Mexican Style Hominy'],
 ['Mexican Style Hominy', 'Verde Salsa'],
 ['Mexican Style Hominy', 'Israeli Style Gefilte Fish'],
 ['Mexican Style Hominy', 'Mild Red Enchilada Sauce'],
 ['Garlic Mint Cheese'],
 ['Spinach Dal Indian Cuisine', 'Madras Lentils Indian Cuisine

In [11]:
max_trans_len = 0
max_trans = []
for trans in transactions_db:
    if len(trans) > max_trans_len:
        max_trans_len = len(trans)
        max_trans = trans

max_trans # Транзакция с наибольшим количеством товаров

['Spinach Dal Indian Cuisine',
 'Madras Lentils Indian Cuisine',
 'Punjab Eggplant',
 'Indian Cuisine Bengal Lentils',
 'Channa Masala Indian Cuisine',
 'Bombay Potatoes Vegetarian',
 'Bombay Lentils',
 '1 Step Kashmir Spinach Indian Cuisine',
 'Jaipur Vegetables']

## 5. Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите три наиболее популярных товара и определите количество покупок (транзакций) этих товаров.

In [12]:
import mlxtend
from mlxtend.preprocessing import TransactionEncoder

In [13]:
te = TransactionEncoder()
transactions_db_bin = te.fit(transactions_db).transform(transactions_db)
_bin = pd.DataFrame(transactions_db_bin, columns=te.columns_)
_bin

,1 Step Kashmir Spinach Indian Cuisine,100% Pure Sesame Seed Oil,100% Whole Soy Organic Gluten Free Tamari Soy Sauce,100% Whole Wheat No Salt Added Matzos,37% Less Sodium Soy Sauce,Agar Agar Sea Vegetable Flakes,All Natural Won Ton Wraps,All Purpose Soy Sauce,"Asian Sauces, Gourmet, Thai Peanut",Asian Vegetable Ramen,...,White Taco Shells,Whole Grain Matzo Meal,Whole Japones Chile Bag,Wide Noodles,Won Ton Wrappers,Yamaimo Soba Buckwheat Noodles,Yellow Corn Taco Dinner Kit,Yellow Corn Taco Shells,Yellow Corn Taco Shells - 12 CT,Yuzu Ponzu Dressing
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1071,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1072,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1073,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
top_products = {}
for col in _bin.columns:
    top_products[col] = _bin[col].sum()
top_products = dict(sorted(top_products.items(), key=lambda item: item[1]))
top_1_popular_product = top_products[list(top_products.keys())[-1]]
top_2_popular_product = top_products[list(top_products.keys())[-2]]
top_3_popular_product = top_products[list(top_products.keys())[-3]]
print(f'Топ-3 популярных товаров:\n1.{list(top_products.keys())[-1]}: {top_1_popular_product}\n2.{list(top_products.keys())[-2]}: {top_2_popular_product}\n3.{list(top_products.keys())[-3]}: {top_3_popular_product}')

Топ-3 популярных товаров:
1.Organic Sea Salt Roasted Seaweed Snacks: 48
2.Taco Seasoning: 43
3.Sriracha Chili Sauce: 24


## 6. При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

In [15]:
from mlxtend.frequent_patterns import fpmax

absolute_support = 2
relative_support = absolute_support / len(_bin)

itemsets_fpm = fpmax(_bin, min_support=relative_support, use_colnames=True)
itemsets_fpm

,support,itemsets
0,0.001860,"(Taco Seasoning, Yellow Corn Taco Shells - 12 CT)"
1,0.001860,(Lemongrass Basil Simmer Sauce)
2,0.001860,(Pad See Ew Noodles)
3,0.001860,(Pad Thai Rice Noodles)
4,0.001860,(Key Lime Cilantro Taco Skillet Sauce with Roa...
...,...,...
212,0.001860,"(Roasted Sesame Seaweed Snacks, Organic Sea Sa..."
213,0.016744,(Organic Spicy Taco Seasoning)
214,0.001860,"(Taco Seasoning, New Mexico Taco Skillet Sauce..."
215,0.022326,(Sriracha Chili Sauce)


In [16]:
longest_itemset = itemsets_fpm.iloc[[0]]
longest_itemset_len = 0
for i, itemset in enumerate(itemsets_fpm['itemsets']):
    if len(itemset) > longest_itemset_len:
        longest_itemset_len = len(itemset)
        longest_itemset = itemsets_fpm.iloc[[i]]
longest_itemset['itemsets']

104    (Spinach Dal Indian Cuisine, Indian Cuisine Be...
Name: itemsets, dtype: object

## 7. Используя пакет mlxtend или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.65.

In [17]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [18]:
itemsets_fpm = apriori(_bin, min_support=0.001, use_colnames=True)
association_table = association_rules(itemsets_fpm, metric="confidence", min_threshold=0.65)
print(association_table)

                                          antecedents  \
0                     (Indian Cuisine Bengal Lentils)   
1             (1 Step Kashmir Spinach Indian Cuisine)   
2             (1 Step Kashmir Spinach Indian Cuisine)   
3             (1 Step Kashmir Spinach Indian Cuisine)   
4                                    (Mushroom Ramen)   
5                        (Bombay Potatoes Vegetarian)   
6                     (Indian Cuisine Bengal Lentils)   
7                      (Channa Masala Indian Cuisine)   
8                     (Indian Cuisine Bengal Lentils)   
9                     (Indian Cuisine Bengal Lentils)   
10                                  (Vegetable Korma)   
11                  (Teriyaki Roasted Seaweed Snacks)   
12                  (Yellow Corn Taco Shells - 12 CT)   
13   (Punjab Eggplant, Indian Cuisine Bengal Lentils)   
14  (Punjab Eggplant, 1 Step Kashmir Spinach India...   
15  (Indian Cuisine Bengal Lentils, 1 Step Kashmir...   
16                    (Indian C

## 8. Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.

In [19]:
association_table[association_table['lift'] == association_table['lift']. max ()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
35,"(Spinach Dal Indian Cuisine, Indian Cuisine Be...","(Punjab Eggplant, 1 Step Kashmir Spinach India...",0.00186,0.00186,0.00186,1.0,537.5,0.001857,inf,1.0
37,"(Spinach Dal Indian Cuisine, 1 Step Kashmir Sp...","(Punjab Eggplant, Indian Cuisine Bengal Lentils)",0.00186,0.00186,0.00186,1.0,537.5,0.001857,inf,1.0
38,"(Punjab Eggplant, Indian Cuisine Bengal Lentils)","(Spinach Dal Indian Cuisine, 1 Step Kashmir Sp...",0.00186,0.00186,0.00186,1.0,537.5,0.001857,inf,1.0
40,"(Punjab Eggplant, 1 Step Kashmir Spinach India...","(Spinach Dal Indian Cuisine, Indian Cuisine Be...",0.00186,0.00186,0.00186,1.0,537.5,0.001857,inf,1.0
